# Анализ музыкальных предпочтений

## Оглавление
<a id = "index"></a>

1. [Задание](#task)
1. [Описание данных](#data_description)
1. [Настройка окружения](#env)
1. [Загрузка и первичное изучение данных](#first_look)
1. [Подготовка данных](#etl)
1. [Главные гипотезы](#main_hypotheses)
1. [Выводы и рекомендации](#conclusion)

## Задание
<a id = "task"></a>
[В оглавление](#index)

## Описание данных
<a id = "data_description"></a>
[В оглавление](#index)

## Настройка окружения
<a id = "env"></a>
[В оглавление](#index)

In [2]:
import pandas as pd

## Загрузка и первичное изучение данных
<a id = "first_look"></a>
[В оглавление](#index)

In [3]:
df = pd.read_csv('music_project.csv')

In [4]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Всего 7 столбцов. Тип данных у каждого столбца - non-null object:
* userID — идентификатор пользователя;
* Track — название трека;  
* artist — имя исполнителя;
* genre — название жанра;
* City — город, в котором происходило прослушивание;
* time — время, в которое пользователь слушал трек;
* Day — день недели.

Количество значений в столбцах различается. В данных есть пропущенные значения.

**Выводы**
- Каждая строка таблицы содержит информацию о композициях определённого жанра в определённом исполнении, которые пользователи слушали в одном из городов в определённое время и день недели. 
- Две проблемы, которые нужно решать: пропуски и некачественные названия столбцов. 
- Для проверки рабочих гипотез особенно ценны столбцы *time*, *day* и *City*. 
- Данные из столбца *genre* позволят узнать самые популярные жанры.

## Подготовка данных
<a id = "etl"></a>
[В оглавление](#index)

Необходимо исключить пропуски, переименовать столбцы и проверить данные на наличие дубликатов.

__Переименование столбцов__

In [8]:
df.columns

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time',
       'weekday'],
      dtype='object')

In [9]:
df.set_axis(
    ['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time', 'weekday'],
    axis='columns',
    inplace=True
)

In [10]:
df.columns

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time',
       'weekday'],
      dtype='object')

#### Проверка пропусков

In [11]:
df.isnull().sum()

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

Для некоторых треков доступна не вся информация. Необходимо заполнение пропусков.

In [12]:
df['track_name'] = df['track_name'].fillna('unknown')

In [13]:
df['artist_name'] = df['artist_name'].fillna('unknown')

In [14]:
df.isnull().sum()

user_id           0
track_name        0
artist_name       0
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

In [15]:
# удаление пустых значений
df.dropna(subset = ['genre_name'],
inplace = True)

In [16]:
df.isnull().sum()

user_id        0
track_name     0
artist_name    0
genre_name     0
city           0
time           0
weekday        0
dtype: int64

#### Проверка дубликатов

In [17]:
df.duplicated().sum()

3755

In [18]:
df = df.drop_duplicates().reset_index(drop=True)

In [19]:
df.duplicated().sum()

0

Дубликаты могли появиться вследствие сбоя в записи данных. 

In [23]:
genres_list = df['genre_name'].unique()

In [24]:
# функция find_genre() для поиска неявных дубликатов в столбце с жанрами

def find_genre(genre):
    counter = 0
    for row in genres_list:
        if row == genre:
            counter += 1
    return counter

Вызов функции *find_genre()* для поиска различных вариантов названия жанра хип-хоп в таблице.

Правильное название — *hiphop*. Другие варианты:
* hip
* hop
* hip-hop

In [25]:
# проверка наличия варианта 'hip'
find_genre('hip')

1

In [26]:
# проверка наличия варианта 'hop'
find_genre('hop')

0

In [27]:
# проверка наличия варианта 'hip-hop'
find_genre('hip-hop')

0

Объявим 
Так исправляем все варианты написания, которые выявила проверка.

In [28]:
# функция заменяет неправильное название этого жанра в столбце *'genre_name'* на *'hiphop'* и проверяет успешность выполнения замены.

def find_hip_hop(df, wrong):
    df['genre_name'] = df['genre_name'].replace(wrong, 'hiphop')
    genre_wrong_count = df.loc[df.loc[:,'genre_name'] == wrong]['genre_name'].count()
    #genre_pop = genre_fight.loc[genre_fight.loc[:, 'genre'] == 'pop']['genre'].count()
    return genre_wrong_count

In [29]:
# замена неверного варианта на hiphop
find_hip_hop(df, 'hip')

0

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60126 entries, 0 to 60125
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      60126 non-null  object
 1   track_name   60126 non-null  object
 2   artist_name  60126 non-null  object
 3   genre_name   60126 non-null  object
 4   city         60126 non-null  object
 5   time         60126 non-null  object
 6   weekday      60126 non-null  object
dtypes: object(7)
memory usage: 3.2+ MB


**Вывод:** На этапе предобработки в данных обнаружились не только пропуски и проблемы с названиями столбцов, но и всяческие виды дубликатов. Их удаление позволит провести анализ точнее. Сведения о жанрах важно сохранить для анализа, не просто удалить все пропущенные значения, но заполнить пропущенные имена исполнителей и названия треков. Имена столбцов теперь корректны и удобны для дальнейшей работы.

## Главные гипотезы
<a id = "main_hypotheses"></a>
[В оглавление](#index)

### Слушают ли музыку в разных городах по-разному?

- __Гипотеза:__ в Москве и Санкт-Петербурге пользователи слушают музыку по-разному. 
- Проверка предположения по данным о трёх днях недели — понедельнике, среде и пятнице.
- Группировка данных по городу, подсчет композиций, для которых известен жанр.

In [32]:
df.groupby('city')['genre_name'].count()

city
Moscow              41892
Saint-Petersburg    18234
Name: genre_name, dtype: int64

- В Москве прослушиваний больше, чем в Питере, но это не значит, что Москва более активна. 
- У Яндекс.Музыки в целом больше пользователей в Москве, поэтому величины сопоставимы.

Подсчет прослушанных в понедельник, среду и пятницу композиции, для которых известен жанр.

In [33]:
df.groupby('weekday')['genre_name'].count()

weekday
Friday       21482
Monday       20866
Wednesday    17778
Name: genre_name, dtype: int64

Понедельник и пятница — время для музыки; по средам пользователи немного больше вовлечены в работу.

Функция *number_tracks()*, принимает как параметры таблицу, день недели и название города, а возвращает количество прослушанных композиций, для которых известен жанр.

In [34]:
def number_tracks(df, day, city):
    track_list = df[ (df['weekday'] == day) & (df['city'] == city)]
    track_list_count = track_list['genre_name'].count()
    return track_list_count

In [35]:
# список композиций для Москвы в понедельник
number_tracks(df, 'Monday', 'Moscow')

15347

In [36]:
# список композиций для Санкт-Петербурга в понедельник
number_tracks(df, 'Monday', 'Saint-Petersburg')

5519

In [37]:
# список композиций для Москвы в среду
number_tracks(df, 'Wednesday', 'Moscow')

10865

In [38]:
# список композиций для Санкт-Петербурга в среду
number_tracks(df, 'Wednesday', 'Saint-Petersburg')

6913

In [39]:
# список композиций для Москвы в пятницу
number_tracks(df, 'Friday', 'Moscow')

15680

In [40]:
# список композиций для Санкт-Петербурга в пятницу
number_tracks(df, 'Friday', 'Saint-Petersburg')

5802

Свод информации в одну таблицу, где ['city', 'monday', 'wednesday', 'friday'] названия столбцов.

In [41]:
# таблица с полученными данными
data1 = [['Moscow', 15347, 10865, 15680],
       ['Saint-Petersburg', 5519, 6913, 5802]]
columns1 = ['city','monday','wednesday','friday']

table = pd.DataFrame(data = data1, columns = columns1)

**Вывод:** результаты показывают, что относительно среды музыку в Петербурге и Москве слушают «зеркально»: в Москве пики приходятся на понедельник и пятницу, а в среду время прослушивания снижается. Тогда как в Санкт-Петербурге среда — день самого большого интереса к музыке, а в понедельник и пятницу он меньше, причём почти одинаково меньше.

### Разная ли музыка в утро понедельника и вечер пятницы

Есть предположение, что в понедельник утром пользователи слушают больше бодрящей музыки (например, жанра поп), а вечером пятницы — больше танцевальных (например, электронику).

Таблицы данных по Москве *moscow_general* и по Санкт-Петербургу *spb_general*.

In [42]:
moscow_general = df.loc[df.loc[:,'city'] == 'Moscow']
spb_general = df.loc[df.loc[:,'city'] == 'Saint-Petersburg']

Функция *genre_weekday()* возвращает список жанров по запрошенному дню недели и времени суток с такого-то часа по такой-то.

In [43]:
def genre_weekday(df, day, time1, time2):
    genre_list = df.loc[(df.loc[:, 'weekday'] == day) & (df.loc[:, 'time'] > time1) & (df.loc[:, 'time'] < time2)]
    genre_list_sorted = genre_list.sort_values(by = 'genre_name', ascending = False).head(10)
    return genre_list_sorted

Cравнение полученных результатов по таблице для Москвы и Санкт-Петербурга в понедельник утром (с 7 до 11) и в пятницу вечером (с 17 до 23).

In [44]:
# вызов функции для утра понедельника в Москве (вместо df таблица moscow_general)
genre_weekday(moscow_general, 'Monday', '07:00:00', '11:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
27442,D7FB50DA,Drumming Circle,Professor Trance,worldbeat,Moscow,09:30:47,Monday
20384,F0C5DA9A,Hasta Siempre Comandante,Juan M. Chazarreta,world,Moscow,08:10:09,Monday
36691,C3B36125,Brava la cuoca,Roberto Scaglioni,world,Moscow,09:51:28,Monday
54812,F37EC6D1,Penjereden Dash Gelir,Chingiz Sadykhov,world,Moscow,08:29:31,Monday
58660,59CBCCE5,Dyolmano Dyulbero,The Bulgarian Voices,world,Moscow,09:53:19,Monday
7388,62E1B964,Child of a Blind Man,Hazmat Modine,world,Moscow,09:37:22,Monday
21143,FF3FD2BD,Ire,Adekunle Gold,world,Moscow,08:26:19,Monday
49652,FE3A4C70,Ма царды 'мбал (Спутник жизни),Оксана Джелиева,world,Moscow,08:53:25,Monday
26801,FDF6D919,Nuage,Jim Cole & Spectral Voices,world,Moscow,08:42:04,Monday
40820,3D844B0C,Tango,Athens Symphony Orchestra,world,Moscow,09:45:53,Monday


In [45]:
# вызов функции для утра понедельника в Петербурге
genre_weekday(spb_general, 'Monday', '07:00:00', '11:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
30068,692AEC35,Baiana,Emicida,world,Saint-Petersburg,09:24:38,Monday
42316,6B62E9CE,The Color of The Night,L' Amore Orchestra,world,Saint-Petersburg,08:56:04,Monday
25542,F018BD36,Aicha,Khaled,world,Saint-Petersburg,09:24:54,Monday
25368,888CB0C0,Sichou Zhi Lu,Ethno Music Orchestra,world,Saint-Petersburg,08:49:11,Monday
44109,2DD633B3,Parole parole,unknown,world,Saint-Petersburg,08:50:08,Monday
45091,B5B6FE0A,Sade,Badmos,world,Saint-Petersburg,09:03:38,Monday
21206,F33FA8C,Pivni Tkach posh edit light,Katya Chilly,world,Saint-Petersburg,08:51:27,Monday
20813,4D2A996C,Closer to Heaven,Mysteria,world,Saint-Petersburg,09:30:23,Monday
19541,CA471C2,The Chieri by Avalon Rising,Avalon Rising,world,Saint-Petersburg,09:05:43,Monday
18608,FEA172A3,Jenny Chicken,Kate Mcmayland and Her Orchestra,world,Saint-Petersburg,09:04:32,Monday


In [92]:
# <вызов функции для вечера пятницы в Москве>
genre_weekday(moscow_general, 'Friday', '17:00:00', '23:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
43694,917047E8,The Nomad,Niraj Chag Featuring Melissa Baten,world,Moscow,21:58:43,Friday
30922,3DC9EF93,Aprum Em Keznov,Arabo Ispiryan,world,Moscow,20:07:26,Friday
31820,9F87ADDC,Bir Gozal,unknown,world,Moscow,20:14:37,Friday
52603,4C61D12D,Apple Penne,Bharathvaj,world,Moscow,21:32:05,Friday
31514,5366E775,Die Reblaus,Hans Moser,world,Moscow,21:40:41,Friday
31317,CAAC4375,Kervansaray,Muhammet İrmikçi,world,Moscow,20:22:48,Friday
2878,2487DD6F,Макарий,Абстрактор,world,Moscow,20:57:45,Friday
14375,BCEA335,Yo Soy Maria,Panterinya,world,Moscow,21:11:20,Friday
31013,77BE1010,Kandıramazsın,Genco Ecer,world,Moscow,22:00:25,Friday
52657,6214F479,Forró do Rei,Trio Virgulino,world,Moscow,20:28:05,Friday


In [46]:
# вызов функции для вечера пятницы в Питере
genre_weekday(spb_general, 'Friday', '17:00:00', '23:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
54324,54A01EAC,Batwanes Beek Ta'a Ninsa,emad batayeh,world,Saint-Petersburg,20:58:18,Friday
29985,47E5088,Что же делать?,Группа «Домбай»,world,Saint-Petersburg,21:15:04,Friday
44531,873D8A1B,Orkestarsko Oro,Maleševski Melos,world,Saint-Petersburg,20:58:55,Friday
18013,1D507F3F,A Cochabamba Me Voy,Daniele Sepe,world,Saint-Petersburg,21:50:57,Friday
48238,F3FB8127,Dari Lolo,Aravod,world,Saint-Petersburg,21:48:19,Friday
22457,A38F488E,The Forest,Taiko Drums,world,Saint-Petersburg,21:27:03,Friday
3470,86F2380D,Королева,Мануэль Исаков,world,Saint-Petersburg,21:19:05,Friday
28527,807DDAF6,El Forga Moura,Haim,world,Saint-Petersburg,21:11:00,Friday
38883,BF27088D,Let It Out - Rafijah & Niyo Rah,Rafijah & Niyo Rah,world,Saint-Petersburg,21:28:56,Friday
29561,DAE710B3,Les nuits de patrimonio,Jérôme Ciosi,world,Saint-Petersburg,20:07:58,Friday


- Популярные жанры в понедельник утром в Питере и Москве оказались похожи: везде, как и предполагалось, популярен поп. 
- Несмотря на это, концовка топ-10 для двух городов различается: в Питере в топ-10 входит джаз и русский рэп, а в Москве жанр *world*.
- В конце недели ситуация не меняется. Поп-музыка всё так же на первом месте. Опять разница заметна только в концовке топ-10, где в Питере пятничным вечером тоже присутствует жанр *world*.

**Вывод:** жанр поп безусловный лидер, а топ-5 в целом не различается в обеих столицах. При этом видно, что концовка списка более «живая»: для каждого города выделяются более характерные жанры, которые действительно меняют свои позиции в зависимости от дня недели и времени.

### Направления в музыке в Москве и Питере

__Гипотеза:__ Питер богат своей рэп-культурой, поэтому это направление там слушают чаще, а Москва — город контрастов, но основная масса пользователей слушает попсу.

Группировка таблицы *moscow_general* по жанру.

In [48]:
moscow_genres = moscow_general.groupby('genre_name')['genre_name'].count().sort_values(ascending = False)
print(moscow_genres.head(10))

genre_name
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre_name, dtype: int64


Группировка таблицы *spb_general* по жанру.

In [49]:
spb_genres = spb_general.groupby('genre_name')['genre_name'].count().sort_values(ascending = False)
print(spb_genres.head(10))

genre_name
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre_name, dtype: int64


**Вывод:** в Москве, кроме абсолютно популярного жанра поп, есть направление русской популярной музыки. Значит, что интерес к этому жанру шире. А рэп, вопреки предположению, занимает в обоих городах близкие позиции.

## Выводы и рекомендации
<a id = "conclusion"></a>
[В оглавление](#index)

Рабочие гипотезы:

* музыку в двух городах — Москве и Санкт-Петербурге — слушают в разном режиме;

* списки десяти самых популярных жанров утром в понедельник и вечером в пятницу имеют характерные отличия;

* население двух городов предпочитает разные музыкальные жанры.

**Общие результаты**

Москва и Петербург сходятся во вкусах: везде преобладает популярная музыка. При этом зависимости предпочтений от дня недели в каждом отдельном городе нет — люди постоянно слушают то, что им нравится. Но между городами в разрезе дней неделей наблюдается зеркальность относительно среды: Москва больше слушает в понедельник и пятницу, а Петербург наоборот - больше в среду, но меньше в понедельник и пятницу.

В результате первая гипотеза < укажите подтверждена/не подтверждена>, вторая гипотеза < укажите подтверждена/не подтверждена > и третья < укажите подтверждена/не подтверждена >.